In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/"

# CSV Dosyalarını Yükle
try:
    # Kredi Detayları
    df_loan = pd.read_csv(base_path + "loan_base_cleaned.csv", low_memory=False)
    # Borçlu Bilgileri
    df_borrower = pd.read_csv(base_path + "borrower_info_cleaned.csv", low_memory=False)
except FileNotFoundError:
    print("Hata: Dosyalar bulunamadı, dosya yollarını kontrol et.")

# Ön Hazırlık: ID Temizliği
# Merge işleminin hatasız olması için ID'leri string yapıp boşlukları siliyoruz
df_loan["id"] = df_loan["id"].astype(str).str.strip()
df_borrower["id"] = df_borrower["id"].astype(str).str.strip()

# BİRLEŞTİRME 
# Temizlenmiş iki tabloyu 'id' üzerinden tek bir DEV tablo yapıyoruz
print("Tablolar birleştiriliyor...")

df_eda = pd.merge(df_loan, df_borrower, on='id', how='inner')

print(f"Birleştirilmiş Veri Seti Boyutu: {df_eda.shape}")
print("-" * 30)
print(f"İlk 5 Sütun: {df_eda.columns[:5].tolist()}")


In [ ]:
# FEATURE ENGINEERING (Yeni Özellikler Türetme)
#Aylık Gelir
df_eda['monthly_inc'] = df_eda['annual_inc'] / 12
# B) Ödeme Yükü
# Kişinin aylık maaşının yüzde kaçı bu kredinin taksitine gidiyor?
# Örn: 5000$ maaşı var, 1000$ taksit ödüyorsa yükü %20'dir.
df_eda['payment_burden'] = (df_eda['installment'] / df_eda['monthly_inc']) * 100


In [ ]:
# RİSK ANALİZİ HAZIRLIĞI 
valid_statuses = ['Fully Paid', 'Charged Off']
df_closed = df_eda[df_eda['loan_status'].isin(valid_statuses)].copy()

# Hedef Değişkeni (Target) Oluşturma
# 1: Battı (Riskli), 0: Ödedi (Temiz)
df_closed['risk_target'] = df_closed['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

print(f"Risk Analizi Yapılacak (Biten) Kredi Sayısı: {df_closed.shape[0]}")


In [ ]:
# GÖRSELLEŞTİRME (EDA)
sns.set_style('whitegrid')
plt.figure(figsize=(22, 18))

# --- GRAFİK 1: Kredi Notu (Grade) ve Batık İlişkisi ---
plt.subplot(3, 2, 1)
# Grade'e göre gruplayıp ortalama risk oranını buluyoruz
risk_by_grade = df_closed.groupby('grade')['risk_target'].mean().reset_index()
sns.barplot(x='grade', y='risk_target', data=risk_by_grade, palette='Reds')
plt.title('Hangi Kredi Notu Daha Riskli?', fontsize=14, fontweight='bold')
plt.ylabel('Batık Oranı (Default Rate)')
plt.axhline(df_closed['risk_target'].mean(), color='black', linestyle='--', label='Genel Ortalama')
plt.legend()

# --- GRAFİK 2: Eyalet Bazlı Risk ---
plt.subplot(3, 2, 2)
# En çok kredi verilen 10 eyalet
top_states = df_closed['state_description'].value_counts().nlargest(10).index
df_top_states = df_closed[df_closed['state_description'].isin(top_states)]
# Risk sıralaması
state_risk = df_top_states.groupby('state_description')['risk_target'].mean().sort_values(ascending=False)
sns.barplot(y=state_risk.index, x=state_risk.values, palette='coolwarm')
plt.title('En Çok Kredi Alan 10 Eyaletin Risk Sıralaması', fontsize=14, fontweight='bold')
plt.xlabel('Batık Oranı')
plt.ylabel('')

# --- GRAFİK 3: Vade (Term) Süresine Göre Risk ---
plt.subplot(3, 2, 3)
sns.barplot(x='term', y='risk_target', data=df_closed, palette='Blues_d')
plt.title('Vade Süresi (36 vs 60 Ay) Riski Nasıl Etkiliyor?', fontsize=14, fontweight='bold')
plt.ylabel('Batık Oranı')

# --- GRAFİK 4: Ödeme Yükü---
plt.subplot(3, 2, 4)
# %50'nin altındaki makul verileri görselleştirelim
sns.kdeplot(data=df_closed[df_closed['payment_burden'] < 50], x='payment_burden', hue='loan_status',
            fill=True, palette={'Fully Paid': 'green', 'Charged Off': 'red'}, alpha=0.3)
plt.title('Ödeme Yükü Analizi (Taksit / Maaş)', fontsize=14, fontweight='bold')
plt.xlabel('Maaşın Yüzde Kaçı Taksite Gidiyor?')

# --- GRAFİK 5: Ev Sahipliği ve Risk ---
plt.subplot(3, 2, 5)
main_home = df_closed[df_closed['home_ownership'].isin(['Rent', 'Mortgage', 'Own'])]
sns.barplot(x='home_ownership', y='risk_target', data=main_home, palette='viridis')
plt.title('Ev Sahipliği Durumuna Göre Risk', fontsize=14, fontweight='bold')
plt.ylabel('Batık Oranı')

# --- GRAFİK 6: Korelasyon Matrisi ---
plt.subplot(3, 2, 6)
corr_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'payment_burden', 'emp_length_numeric', 'risk_target']
corr = df_closed[corr_cols].corr()
sns.heatmap(corr, annot=True, cmap='RdBu_r', fmt='.2f', vmin=-1, vmax=1)
plt.title('Değişkenler Arası İlişki (Korelasyon)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()